In [ ]:
import pymCADRE
import pandas as pd
import numpy as np
import gurobipy
from pymCADRE.rank.rank_reactions import *
from pymCADRE.prune.prune_model import *
from cobra.io import read_sbml_model, write_sbml_model
import os

from cobra.flux_analysis import fastcc

In [ ]:
# set paht for Gurobi License
os.environ['GUROBI_HOME'] = "/home/jiang817/cybergut_project/Software/gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/home/jiang817/cybergut_project/Software/gurobi_license/gurobi.lic"

In [ ]:
# Load Recon3D
model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

In [ ]:
# Set exchagne reaction boundries as (-100, 100)
for i in range (len(model.reactions)):
    if model.reactions[i].id.startswith('EX_'):
        model.reactions[i].bounds = (-100, 100)

In [ ]:
# Load gene ID
G = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_pymCADRE_INPUT/ENTRZE_ID_RECON3D_XML.csv')
G = list(G['ENTREZ_GENE_ID'])


In [ ]:
# Load Ubuquity scores of genes
U = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_pymCADRE_INPUT/Ubiquity_Score_RECON3D_XML.csv')
U = list(U['Ubiquity_score'])


In [ ]:
# Load confidence scores
confidence_scores = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_pymCADRE_INPUT/Recon3D_XML_confidence_scores.csv')
confidence_scores = np.float64(list(confidence_scores['Confidence Score']))

In [ ]:
# Run rank reactions
GM, C, NC, P, Z, model_C = rank_reactions(model, G, U, confidence_scores, [], 2)   

In [ ]:
# Set the formula of metabolite -'protein_c' as 'PROTEIN', to avoid error 
for i in range(len(GM.metabolites)):
    if GM.metabolites[i].id == 'protein_c':
        GM.metabolites[i].formula = 'PROTEIN'

In [ ]:
# Run prune_model
try:
    
    print('Pruning reactions...')
    t0 = process_time()
    PM, cRes = prune_model(GM, P, C, Z, 1/3, '/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_XML_pymCADRE_INPUT/KeyMets_Recon3D_XML.txt', 1, 2)
    # Stop the stopwatch / counter
    t_stop = process_time()
    # compute elapsed time
    pruneTime = t_stop - t0
    
    print('PASS...', 'Function prune_model ran without error')

except (RuntimeError, TypeError, NameError) as inst:

    print('FAIL...', 'Function prune_model was terminated with the error:')
    print(type(inst))
    print(inst.args)

In [ ]:
write_sbml_model(model, '/home/jiang817/cybergut_project/Draft_model/Draft_model_with_pymCADRE/test.xml')

In [ ]:
# run fastcc to obtain consistent reconstruction 
model_fastcc = fastcc(model)

# save consistent reconstuction
write_sbml_model(model_fastcc, '/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_pymcadre_fastcc.xml')